In [10]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pickle
from scipy.stats import pearsonr
from scipy.io import loadmat

### Spectral selectivity comparison

In [2]:
with open("SC_comps.pkl", "rb") as f:
    SC_comp = pickle.load(f)

In [3]:
pearsonr(SC_comp['sim_SCs'], SC_comp['data_SCs'], alternative='greater')

PearsonRResult(statistic=0.9587100848473007, pvalue=9.350962532117455e-24)

### Photocurrent trace

In [4]:
# Load the stimuli and data
data_path = "../../../data/RiekeMouseConeResponses.mat"
data = loadmat(data_path)

responses = [data['ConeResponses'][0][i][0][0][0][0] for i in range(6)]
stimuli = [data['ConeResponses'][0][i][0][0][1][0] for i in range(6)]

test_stim = stimuli[4]
test_response = responses[4]

# Load the fits
with open("cascade_final_fit.pkl", "rb") as f:
    cascade_fit = pickle.load(f)
trained_normed = cascade_fit['trained_normed']
init_normed = cascade_fit['init_normed']

In [5]:
np.var(test_response)

0.06765989907710937

In [6]:
MSE = np.mean((test_response - trained_normed) ** 2)
MSE

Array(0.00878756, dtype=float32)

In [7]:
1 - (MSE / np.var(test_response))

Array(0.87012166, dtype=float32)

### Glutamate traces

In [11]:
# Load the data
with open("../../../data/120_center_responses.pkl", "rb") as f:
    trace_selection = pickle.load(f) 

In [12]:
# Load all the cell numbers
with open("../../../params/ribbon/cell_nums.txt", "r") as f:
    cell_nums = json.load(f)

# Load the final solutions
with open("ribbon_solns.pkl", "rb") as f:
    final_solns = pickle.load(f)

In [15]:
n2show = 5

# Choose a random subset from the best 20 selection that you use for mnist
with open("../../../params/ribbon/best_20_params_w_cellnums.pkl", "rb") as f:
    best_cell_nums, best_params = pickle.load(f)
_ = np.random.seed(0)
rand_dorsal = np.random.choice(best_cell_nums[:10], n2show, replace=False)
rand_ventral = np.random.choice(best_cell_nums[10:], n2show, replace=False)
trace_indices = np.concatenate((rand_dorsal, rand_ventral))
trace_indices

array([ 6, 14, 11, 20,  5, 62, 66, 61, 65, 75])

In [20]:
rs = []
mses = []
for i, iin in enumerate(best_cell_nums):
    # Plot the simulation results
    data_time = np.arange(0, 4, step=1/500)
    soln_idx = list(cell_nums).index(iin) # instead of i if calced all final solns
    # plt.plot(data_time, final_solns[soln_idx], c="k")

    trace_selection_g = trace_selection[iin, :1000]
    g_time = data_time[:1000]
    trace_selection_uv = trace_selection[iin, 1000:]
    uv_time = data_time[1000:]

    full_data = np.concatenate((trace_selection_g, trace_selection_uv))
    # plt.plot(data_time, full_data, c="g")

    MSE = np.mean((final_solns[soln_idx] - full_data) ** 2)
    mses.append(MSE)
    r = pearsonr(final_solns[soln_idx], full_data)[0]
    rs.append(r)

rs, mses


([0.9476932,
  0.89898777,
  0.9364698,
  0.8761715,
  0.88651097,
  0.9429598,
  0.89325583,
  0.9396544,
  0.9271328,
  0.87048376,
  0.97316915,
  0.95417106,
  0.9758342,
  0.94728297,
  0.91283834,
  0.9756392,
  0.90819657,
  0.96493465,
  0.8766454,
  0.8286468],
 [Array(0.08899015, dtype=float32),
  Array(0.12138499, dtype=float32),
  Array(0.02489917, dtype=float32),
  Array(0.1610046, dtype=float32),
  Array(0.1021454, dtype=float32),
  Array(0.09049678, dtype=float32),
  Array(0.10857911, dtype=float32),
  Array(0.11733963, dtype=float32),
  Array(0.08683628, dtype=float32),
  Array(0.13966048, dtype=float32),
  Array(0.05644567, dtype=float32),
  Array(0.12068921, dtype=float32),
  Array(0.05416939, dtype=float32),
  Array(0.4057353, dtype=float32),
  Array(0.05863358, dtype=float32),
  Array(0.06829395, dtype=float32),
  Array(0.22252695, dtype=float32),
  Array(0.19774127, dtype=float32),
  Array(0.21898669, dtype=float32),
  Array(0.43664518, dtype=float32)])

In [21]:
np.min(rs), np.max(rs)

(0.8286468, 0.9758342)

In [22]:
np.min(mses), np.max(mses)

(0.024899172, 0.43664518)